## CEX-DEX Arbitrage Research

Use this Notebook as the starting point for your CEX-DEX arbitrage research.

In [1]:
import os
import asyncio
import nest_asyncio
import aioprocessing
from functools import partial
from dotenv import load_dotenv

load_dotenv(override=True)

HTTP_RPC_URL = os.getenv('HTTP_RPC_URL')
WS_RPC_URL = os.getenv('WS_RPC_URL')

#### Utils

A function for handling websocket connection errors. Wrapping data streaming functions with **reconnecting_websocket_loop** will automatically reconnect everytime a connection error occurs.

In [2]:
from utils import (
    reconnecting_websocket_loop,
    calculate_next_block_base_fee,
)

#### CEX data streams

Currently implemented for Binance websocket stream of depth5 data.

In [3]:
from cex_streams import (
    stream_binance_usdm_orderbook,
    stream_okx_usdm_orderbook,
)

#### DEX data streams

In [4]:
from dex_streams import (
    stream_new_blocks,
    stream_uniswap_v2_events,
)

#### Aggregator

In [5]:
from aggregator import aggregate_cex_orderbooks

#### Event handler

In [6]:
import datetime
from spread_chart import Publisher

async def cex_event_handler(port: int, event_queue: aioprocessing.AioQueue):   
    pub = Publisher(port)

    orderbooks = {}
    
    last_updated = datetime.datetime.now()
    
    while True:
        data = await event_queue.coro_get()
        
        symbol = data['symbol']
        
        if data['source'] == 'cex':
            if symbol not in orderbooks:
                orderbooks[symbol] = {}
                
            orderbooks[symbol][data['exchange']] = data
            multi_orderbook = aggregate_cex_orderbooks(orderbooks[symbol])
            
            now = datetime.datetime.now()
            if (now - last_updated).total_seconds() > 1:
                best_bid = multi_orderbook['bids'][0][0]
                best_ask = multi_orderbook['asks'][0][0]
                bid_ask_spread = (best_bid / best_ask - 1) * 100
                await pub.send({'spread': bid_as_spread})
                
                last_updated = now
                print(f'[{now}] Spread: {round(bid_ask_spread, 3)}%')

#### Run streams

In [7]:
nest_asyncio.apply()

# define an event_queue to publish realtime data
event_queue = aioprocessing.AioQueue()

symbols = ['ETH/USDT']

binance_stream = reconnecting_websocket_loop(
    partial(stream_binance_usdm_orderbook, symbols, event_queue),
    tag='binance_stream'
)

okx_stream = reconnecting_websocket_loop(
    partial(stream_okx_usdm_orderbook, symbols, event_queue),
    tag='okx_stream'
)

port = 9999
event_handler_loop = cex_event_handler(port, event_queue)

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait([
    binance_stream,
    okx_stream,
    event_handler_loop,
]))

[2023-08-08 16:03:58.625304] Spread: 0.034%
[2023-08-08 16:03:59.695921] Spread: 0.034%
[2023-08-08 16:04:00.707374] Spread: 0.034%
[2023-08-08 16:04:01.709402] Spread: 0.034%
[2023-08-08 16:04:02.733715] Spread: 0.034%
[2023-08-08 16:04:03.865914] Spread: 0.034%
[2023-08-08 16:04:04.920246] Spread: 0.034%
